# Use Case

As a physician, I want to calculate a patient's cardiovascular risk using the Framingham calculator - http://reference.medscape.com/calculator/framingham-coronary-risk-ldl

Several peices of data are needed to perform this calculation. We will use FHIR to pull the data we need from the patient's health record.

In [ ]:
!pip3 install requests

#### First, let's import and define some variables

In [ ]:
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import requests
import json

# Define the FHIR Endpoint
fhir_url = "https://api.logicahealth.org/test231/open/"

# I. "Patient" resource
We will use the `"Patient"` resource to represent demographic information about a patient.

#### TODO
First, perform a GET on patient with id of 'SMART-1627321' using the `requests` library. This returns a `Response` object, which has an attribute called `.text` which contains the actual response we're interested in. Parse this response text using the `json` library to create a Python dictionary.

In [ ]:
patient_url = fhir_url + "Patient/SMART-1627321"
patient_url

In [ ]:
patient_res = ____.get(patient_url)

In [ ]:
patient_res

In [ ]:
print(patient_res.text[:50])

In [ ]:
patient_json = json.loads(patient_res.____)

In [ ]:
patient_json

Now that we have this patient's information as a dictionary, we can access specific attributes about the patient using the dictionary's keys.

#### TODO
Save the first and last names of the patients and store them as variables `first_name` and `last_name`. These values are stored in the `["name"]` field of `patient_json`.

In [ ]:
patient_json["name"]

In [ ]:
# Save the first and last name of the patient
first_name = patient_json["name"][0][____][0]
last_name = ____

In [ ]:
print("Patient's Name = {0} {1}".format(first_name, last_name))

#### TODO
Next, save and print the patient's date of birth and gender. You can print out the dictionary's keys to see which values to use.

In [ ]:
patient_json.keys()

In [ ]:
# Print the patient's date of birth
dob = patient_json[____]
print("DOB = {0}".format(dob))

In [ ]:
# Print patient's gender
gender = ____
print("Gender = {0}".format(gender))

Next, we'll calculate the patient's age using the `rdelta` method of the `dateutil` module. This is similar to when we calculated patient's age at death in Week 6.

In [ ]:
now = datetime.datetime.today()
dob2 = datetime.datetime.strptime(dob, '%Y-%m-%d')
rdelta = relativedelta(now, dob2)
age = rdelta.years

In [ ]:
# print the years from the rdelta object
print("Patient's age = {0}".format(age))

# II. "Observation" resource 
LOINC codes:  
&nbsp;&nbsp;&nbsp;LDL: 18262-6  
&nbsp;&nbsp;&nbsp;HDL: 2085-9

## LDL Value
Similar to what we did for the `Patient` resource above, we will make a request for an `Observation`. This time we'll also pass in a search parameter specifying the LOINC code for an LDL test and for our specific patient. We'll then parse out the value of the test.

In [ ]:
ldl_url = fhir_url + "Observation?code=18262-6&subject=SMART-1627321"
ldl_url

In [ ]:
ldl_res = ____.____(ldl_url)

In [ ]:
ldl_res

In [ ]:
ldl_json = json.loads(____)

In [ ]:
ldl_resource = ldl_json["entry"][0]["resource"]

In [ ]:
ldl_resource

In [ ]:
ldl_resource.keys()

In [ ]:
ldl_val = ldl_resource["valueQuantity"]["value"]
ldl_unit = ldl_resource["valueQuantity"]["unit"]

In [ ]:
print(ldl_val, ldl_unit)

## HDL value
#### TODO
Now, follow the same steps for the HDL lab value. Start by performing a `GET` operation using the provided URL and then parse out the `hdl_value`, similar to what we did above for the LDL.

In [ ]:
hdl_url = fhir_url + "Observation?code=2085-9&subject=SMART-1627321"

In [ ]:
hdl_url

In [ ]:
# Complete this code to get the value and unit for the HDL test
# ...
# ...

hdl_val = ____
hdl_unit = ____
print(hdl_val, hdl_unit)

## Systolic and diastolic blood pressure
Use the "Observation" resource.  
LOINC code: 55284-4

In [ ]:
press_url = fhir_url + "Observation?code=55284-4&subject=SMART-1627321"
press_url

In [ ]:
press_res = ____
press_json =  ____

In [ ]:
press_resource = press_json["entry"][0]["resource"]

In [ ]:
press_resource.keys()

The systolic and diastolic blood pressures are stored in the first and second elements of the `"component"` field.

In [ ]:
sys = press_resource["component"][0]["valueQuantity"]["value"]
dia = press_resource["component"][1]["valueQuantity"]["value"]

In [ ]:
print("Systolic: " + str(sys) + " " + press_resource["component"][0]["valueQuantity"]["unit"])
print("Diastolic: " + str(dia) + " " + press_resource["component"][0]["valueQuantity"]["unit"])

# III. "Condition" resource

### Does the patient have diabetes?
Of course, there are many types of diabetes. Normally, you would call a "ValueSet" resource to get all possible diabetes codes, but to keep things simple, we'll just use the code for Type 2 diabetes.

Use the "Condition" resource using the SNOMED code **44054006**. We'll create a boolean variable called `diabetes` which will be set to `True` if the patient has this SNOMED code and `False` otherwise.

In [ ]:
diabetes = False

In [ ]:
diabetes_url = fhir_url + "Condition?code=44054006&subject=SMART-1627321"
diabetes_url

In [ ]:
diabetes_res = ____
diabetes_json = ____

In [ ]:
if len(diabetes_json["entry"]) > 0:
    diabetes = True

In [ ]:
print("Diabetes: ", diabetes)

### Does the patient smoke?
Again, there are many codes for smoking. First, use the code for "Tobacco smoking status" to get all Observation resources that define the patient's smoking status. Then filter by SNOMED code for "Occasional tobacco smoker".

LOINC "Tobacco smoking status": 72166-2  
SNOMED "Occasional tobacco smoker": 428041000124106

In [ ]:
smoking = False

In [ ]:
smoking_url = fhir_url + "Observation?code=72166-2&subject=SMART-1627321"
smoking_url

In [ ]:
smoke_res = ____
smoke_json = ____

In [ ]:
if len(smoke_json["entry"]) > 0:
    smoke_resource = smoke_json["entry"][0]["resource"]
    if smoke_resource["valueCodeableConcept"]["coding"][0]["code"] == "428041000124106":
        smoking = True

In [ ]:
print("Smoker:", smoking)

### Coronary Risk Calculator

In [ ]:
points = 0

if gender == "male":
    if age <= 34:
        points += -1
    if age >= 40 and age <= 44:
        points += 1
    if age >= 45 and age <= 49:
        points += 2
    if age >= 50 and age <= 54:
        points += 3
    if age >= 55 and age <= 59:
        points += 4
    if age >= 60 and age <= 64:
        points += 5
    if age >= 65 and age <= 69:
        points += 6
    if age >= 70:
        points += 7
    if ldl_val < 100:
        points += -3
    if ldl_val >= 160 and ldl_val <= 189:
        points += 1
    if ldl_val >= 190:
        points += 2
    if hdl_val < 35:
        points += 2
    if hdl_val >= 35 and hdl_val <= 44:
        points += 1
    if hdl_val >= 60:
        points += -1
    if sys <=129 and dia >= 85 and dia <= 89:
        points += 1
    if sys >= 130 and sys <=139 and dia < 90:
        points += 1
    if sys <= 139 and dia >= 90 and dia <= 99:
        points += 2
    if sys >= 140 and sys <=159 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 2
    if smoking == True:
        points += 2

In [ ]:
if gender == "female":
    if age <= 34:
        points += -9
    if age >= 35 and age <= 39:
        points += -4
    if age >= 45 and age <= 49:
        points += 3
    if age >= 50 and age <= 54:
        points += 6
    if age >= 55 and age <= 59:
        points += 7
    if age >= 60:
        points += 8
    if ldl_val < 100:
        points += -2
    if ldl_val >= 160:
        points += 2
    if hdl_val < 35:
        points += 5
    if hdl_val >= 35 and hdl_val <= 44:
        points += 2
    if hdl_val >= 45 and hdl_val <= 49:
        points += 1
    if hdl_val >= 60:
        points += -2
    if sys < 120 and dia < 80:
        points += -3
    if sys <=159 and dia >=90 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 4
    if smoking == True:
        points += 2

In [ ]:
male_results = {0:3,1:4,2:4,3:6,4:7,5:9,6:11,7:14,8:18,9:22,10:27,11:33,12:40,13:47}
female_results = {0:2,1:2,2:3,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:11,11:13,12:15,13:17,14:20,15:24,16:27}

if points >=14 and gender == "male":
    risk = ">= 56%"
elif points >=17 and gender == "female":
    risk = ">= 32%"
elif gender == "male":
    percent = male_results[points]
    risk = str(percent)+"%"
else:
    percent = female_results[points]
    risk = str(percent)+"%"
print("Total number of points: " + str(points))
print("Risk of Heart Disease: " + risk)